## 5 Interesting Pytorch Functions:

Pytorch is an open sourced Machine Learning Library developed on the torch library and maintained by Facebook. Below are 5 of the commonly used pytorch functions which a Deep Learning practitioner should be aware of:

1. torch.tensor.<i><b>numpy()</b></i> :- torch.tensor.numpy() is used to convert a tensor to a numpy array
2. torch.tensor.<i><b>new_tensor()</b></i> :- torch.tensor.new_tensor() is used to create a new tensor from an existing tensor or a data collection.
3. torch.tensor.<i><b>view()</b></i>:- torch.tensor.view() is used to get a view of the existing tensor without explicit memory copy of the tensor data.
4. torch.tensor.<i><b>is_leaf</b></i> :- torch.tensor.is_leaf is used to detect whether a tensor is a leaf Tensor or not.
5. torch.tensor.<i><b>backward()</b></i>:- torch.tensor.backward() calculates the gradient of current tensor w.r.t leaf tensor. 


Below, we will explore the working code examples of each of the above mentioned function in pytorch along with <b>one drawback for each</b>. 


                                        Let's Start!!! 


### Import the required libraries 

In [1]:
import torch
import numpy as np

### Which version of Python is used??

In [2]:
!python --version      ##running version of python

Python 3.7.7


## Let's start exploring the above mentioned 5 functions sequentially:

### torch.tensor.numpy()
    
As intuitive, it looks this function is used to convert a tensor to a numpy multi-dimensional array.      
    

In [3]:
t1 = torch.tensor([20.,30.,40.]) #3x1 tensor
t1

tensor([20., 30., 40.])

In [4]:
n1 = t1.numpy()  #coneverting a 3x1 tensor to a multi-dimensional 3x1 numpy array
n1

array([20., 30., 40.], dtype=float32)

In [5]:
hex(id(t1)),hex(id(n1)) # memory locations  of the defined tensor t1 and derived numpy array n1

('0x7f285da92140', '0x7f285daa3990')

We see that, both the tensors and derived array has different memory allocations.

So far good, <i>any tensor can be converted to a numpy array</i>..<b>Wait is it ?</b> Let's find out !

In [6]:
#Let's define the same tensors with requires_grad as set to True
t1 = torch.tensor([20.,30.,40.],requires_grad=True) 
print(t1)


tensor([20., 30., 40.], requires_grad=True)

<b> When tensor.torch.numpy failed! </b>

In [7]:
n1 = t1.numpy()

RuntimeError: Can't call numpy() on Variable that requires grad. Use var.detach().numpy() instead.

<b>What ???? The tensor did not get converted to a numpy array this time.</b>

This is because pytorch can only convert tensors to numpy arrays which will not be a part of any dynamic computations in pytorch, for e.g : <i>'requires_grad'</i> calculates the gradient of output w.r.t independent tensors(Leaf Variables) like  t1 here and stores them in a dynamic computation graph.

More information on requires_grad can be found in [here](https://pytorch.org/docs/stable/autograd.html?highlight=requires_grad#torch.Tensor.requires_grad)

### torch.tensor.new_tensor()

As intuitive, the new_tensor() method creates a new tensor from an existing pre-defined tensor or a new data collection.The method returns a new Tensor with data as the tensor data. By default, the returned Tensor has the same [torch.dtype](https://pytorch.org/docs/stable/tensor_attributes.html#torch.torch.dtype) and [torch.device](https://pytorch.org/docs/stable/tensor_attributes.html#torch.torch.device) as this tensor.

In [8]:
t1 = torch.ones((2,),dtype=torch.int8)
t2 = t1.new_tensor([1.,2.,3.])
t2

tensor([1, 2, 3], dtype=torch.int8)

As we can see that, t2 is a new tensor with formed with new data but similar to the data types of the tensor t1.

In [9]:
t2 = t1.new_tensor(t1) # here we kind of try to duplicate the t1 tensor

/home/souptik/anaconda3/envs/snowflakes/lib/python3.7/site-packages/ipykernel_launcher.py:1: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than tensor.new_tensor(sourceTensor).
  """Entry point for launching an IPython kernel.


In [10]:
t2 

tensor([1, 1], dtype=torch.int8)

From the above example, we see that t2 is copied from t1, rather a replica of t1 , but we do face a warning saying that the source tensor should be cloned and detached with [.clone()](https://pytorch.org/docs/stable/tensors.html?highlight=clone#torch.Tensor.clone) and [.detach()](https://pytorch.org/docs/stable/autograd.html?highlight=detach#torch.Tensor.detach) respectively. 

Let's see why is it so?

<b>When torch.tensor.new_tensor() failed!</b>

In [11]:
# Create tensors.
x = torch.tensor(1., requires_grad=True )
w = torch.tensor(2., requires_grad=True)
b = torch.tensor(3., requires_grad=True)
x, w, b

(tensor(1., requires_grad=True),
 tensor(2., requires_grad=True),
 tensor(3., requires_grad=True))

In [12]:
y = w*x + b 
y

tensor(5., grad_fn=<AddBackward0>)

In [13]:
y.backward() #calculates the gradient of output w.r.t inputs

In [14]:
#gradients of outputs w.r.t inputs
print('dy/dx :',x.grad)  
print('dy/dw :',w.grad)
print('dy/db :',b.grad)

dy/dx : tensor(2.)
dy/dw : tensor(1.)
dy/db : tensor(1.)


In [15]:
a = x.new_tensor(x,requires_grad=True) #creating a copy of a tensor x

/home/souptik/anaconda3/envs/snowflakes/lib/python3.7/site-packages/ipykernel_launcher.py:1: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than tensor.new_tensor(sourceTensor).
  """Entry point for launching an IPython kernel.


In [16]:
print(a.grad)  #no grad of tensor x is copied to tensor a

None


So, from the linear computation above of y =w*x+ b , we see that we were able to compute the output y, w.r.t x,w,and b; <b>but when the same tensor x was copied to a new tensor a, the gradients associated with x(x.grad), were not transferred to tensor a</b>. 

For dynamic graph computations,the gradients are only associated of outputs w.r.t leaf variables(for e.g: x) only,which are directly involved in getting the output(for e.g: y) 

<b>Hence, torch.tensor.new_tensor() does not copy the gradients of a tensor</b>

### torch.tensor.view()

View tensor shares the same underlying data with its base tensor. Supporting View avoids explicit data copy, thus allows us to do fast and memory efficient reshaping, slicing and element-wise operations.

In [17]:
t1 = torch.randn(2,3)
t1         

tensor([[ 0.2592, -1.1524, -0.4591],
        [ 0.8569,  0.1141, -0.1472]])

In [18]:
t1_ = t1.view(-1) #flattening as per view reshape
t1_

tensor([ 0.2592, -1.1524, -0.4591,  0.8569,  0.1141, -0.1472])

In [19]:
t1_.shape

torch.Size([6])

#### When torch.tensor.view() failed !

<b>torch.tensor.view() does not work for non-contiguous tensors</b>. Non-contiguous tensors are those tensors whose memory allocation of the elements is different from the declared sequence of the elements in the tensor. More information can be found [here](https://discuss.pytorch.org/t/contigious-vs-non-contigious-tensor/30107/2).

In [20]:
t1.is_contiguous() #check if the tensor is contiguous in memory allocation or not

True

In [21]:
print(t1.transpose(0,1))
print(t1.transpose(0,1).is_contiguous())

tensor([[ 0.2592,  0.8569],
        [-1.1524,  0.1141],
        [-0.4591, -0.1472]])
False


In [22]:
t1.transpose(0,1).view(-1)

RuntimeError: view size is not compatible with input tensor's size and stride (at least one dimension spans across two contiguous subspaces). Use .reshape(...) instead.

As we see, here the .view() function did not work for the transposed tensor of tensor t1 as it is a non-contiguous tensor. You can also check the memory allocations of the view tensor and the original tensor using [stride](https://pytorch.org/docs/stable/tensors.html?highlight=stride#torch.Tensor.stride)

### torch.tensor.is_leaf

This method is used to detect if the tensor is a leaf Variable or not. Let's understand with few examples!!!

In [23]:
t1 = torch.randn(2,2,requires_grad=True) #initialise a random 2x2 tensor
t1

tensor([[-0.3277,  0.0036],
        [ 0.7249,  0.1988]], requires_grad=True)

In [24]:
t1.is_leaf

True

#### When torch.tensor.is_leaf failed !

As per official Pytorch Documentation 1.5.0 , For Tensors that have requires_grad which is True, they will be leaf Tensors if they were created by the user. <b>This means that they are not the result of an operation and so grad_fn is None</b>. [Reference](https://pytorch.org/docs/stable/autograd.html?highlight=leaf#torch.Tensor.is_leaf)

In [25]:
t2 = torch.randn(2,2,requires_grad=True)+2
t2

tensor([[3.8632, 4.2549],
        [1.9408, 2.3880]], grad_fn=<AddBackward0>)

As shown above, the tensor t2 is a random tensor with requires_grad set to True but is a result of an operation of adding 2 also. So, in this case it will not be a Leaf Tensor. Let's check!!

In [26]:
t2.is_leaf #t2 is not a leaf Tensor as it is created by an operation with requires_grad = True

False

### torch.tensor.backward()

[torch.tensor.backward](https://pytorch.org/docs/stable/autograd.html?highlight=backward#torch.Tensor.backward) This function is useful for computing the gradient of a tensor w.r.t leaf tensors

In [27]:
t1 = torch.randn(1,1,requires_grad=True)
t2 = torch.randn(1,1,requires_grad=True)


In [28]:
t1,t2

(tensor([[0.7683]], requires_grad=True),
 tensor([[-0.5562]], requires_grad=True))

Above are 2 randomly defined tensors of dimensions 1x1 with requires_grad = True. Let's involve these variables in a linear computation and calculate the gradient of output w.r.t these variables. 

In [29]:
y = t1*t2
y

tensor([[-0.4273]], grad_fn=<MulBackward0>)

In [30]:
y.backward() #compute the gradient of output y w.r.t leaf tensors t1,t2

In [31]:
print(t1.grad),  #dy/dt1
print(t2.grad)   #dy/dt2

tensor([[-0.5562]])
tensor([[0.7683]])


<b>When torch.tensor.backward() failed !</b>

In [32]:
t1 = torch.randn(2,2,requires_grad=True)
t2 = torch.randn(2,2,requires_grad=True)

In [33]:
t1,t2

(tensor([[-0.1056,  1.5187],
         [ 0.1242,  0.1722]], requires_grad=True),
 tensor([[ 1.3428, -0.8163],
         [-0.2947, -0.1649]], requires_grad=True))

Above are 2 randomly defined tensors of dimensions 2x2 with requires_grad = True. Let's involve these variables in a linear computation and calculate the gradient of output w.r.t these variables. 

In [34]:
y = t1*t2
y

tensor([[-0.1418, -1.2397],
        [-0.0366, -0.0284]], grad_fn=<MulBackward0>)

In [35]:
y.backward()

RuntimeError: grad can be implicitly created only for scalar outputs

As seen from the error, <b>torch.tensor.backward() works for a scalar output or a tensor with a unit dimension.</b>
More discussion on this issue can be found [here](https://discuss.pytorch.org/t/loss-backward-raises-error-grad-can-be-implicitly-created-only-for-scalar-outputs/12152).

### Conclusion

Above mentioned are 5 of the commonly used pyTorch functions with their exceptions.


### Reference Links:

The complete documentation of PyTorch Documentation V1.5.0 can be found in: https://pytorch.org/docs/stable/tensors.html

### Share the code to Jovian.ML

In [36]:
# Install the jovian Python library
!pip install jovian --upgrade -q

In [37]:
# Import the library in your Jupyter notebook
import jovian

<IPython.core.display.Javascript object>

In [38]:
# Upload your notebook & get a sharing link with a single command
jovian.commit()

<IPython.core.display.Javascript object>

[jovian] Attempting to save notebook..
[jovian] Updating notebook "souptikmajumder/assignment-1-zero-to-gan" on https://jovian.ml/
[jovian] Uploading notebook..
[jovian] Capturing environment..
[jovian] Committed successfully! https://jovian.ml/souptikmajumder/assignment-1-zero-to-gan


'https://jovian.ml/souptikmajumder/assignment-1-zero-to-gan'